In [1]:
from six.moves import cPickle as pickle

import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

import os as os
import fnmatch
import argparse
import pathlib
import time

import numpy as np
if "DISPLAY" not in os.environ:
    mpl.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import scipy.io as sio
import glob 

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold  # k-fold cross validation
from sklearn.preprocessing import StandardScaler  # data normalization
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.pipeline import Pipeline # model pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics, tree

%reload_ext autoreload

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rc('text', usetex=True)
plt.rc('font', family='times')

plt.rcParams.update({'font.size': 22})
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2



In [2]:
knownNames = glob.glob("../dataset/Dados_SONAR/[A-CH]*.mat")

X = [sio.loadmat(x)['ent_norm'] for x in knownNames]

# print(X)

minLengthVector = [np.shape(x)[0] for x in X]
Y = [x.split('/')[-1][0] for x in knownNames]
minLength = int(np.min(minLengthVector))

le = LabelEncoder()
target = le.fit_transform(Y)

print(target)

# namesA = glob.glob("../dataset/Dados_SONAR/A*")
# X_A = [sio.loadmat(x)['ent_norm'] for x in namesA]
# namesB = glob.glob("../dataset/Dados_SONAR/B*")
# X_B = [sio.loadmat(x)['ent_norm'] for x in namesB]
# namesC = glob.glob("../dataset/Dados_SONAR/C*")
# X_C = [sio.loadmat(x)['ent_norm'] for x in namesC]
# namesH = glob.glob("../dataset/Dados_SONAR/H*")
# X_H = [sio.loadmat(x)['ent_norm'] for x in namesH]

# x2 = X_A.append(X_B)




# minLength = np.zeros(len(names))

# for i in range(len(names)):
#     x = sio.loadmat(str(names[i]))
#     y = x["ent_norm"]
#     minLength[i] = np.shape(y)[0]
# print(np.min(minLength))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 1 2
 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2]


In [34]:
xMatrix = np.stack([x[:minLength,:].reshape(-1) for x in X])

X_train, X_test, y_train, y_test = train_test_split(xMatrix, target, test_size=0.3, random_state=42)

pca = PCA(10)
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)

model = Pipeline([('pca', PCA()), ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))])


In [35]:
fold = KFold(5)


In [36]:
parameters = {'pca__n_components': [1, 2, 3, 4, 5, 6 , 7, 8, 9, 10], 'rf__n_estimators': [100], 'rf__n_jobs': [-1],
              'rf__min_samples_leaf': [1, 10, 20, 30]}
gridsearchcv = GridSearchCV(model, parameters, cv=fold)
gridsearchcv.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('rf', RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_node...stimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'rf__n_jobs': [-1], 'rf__n_estimators': [100], 'rf__min_samples_leaf': [1, 10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
gridsearchcv.grid_scores_

/home/felipe/miniconda2/envs/siemens2.7/lib/python2.7/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.27059, std: 0.04706, params: {'pca__n_components': 1, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.32941, std: 0.12113, params: {'pca__n_components': 1, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.22353, std: 0.07804, params: {'pca__n_components': 1, 'rf__min_samples_leaf': 20, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.36471, std: 0.14600, params: {'pca__n_components': 1, 'rf__min_samples_leaf': 30, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.60000, std: 0.10120, params: {'pca__n_components': 2, 'rf__min_samples_leaf': 1, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.60000, std: 0.11406, params: {'pca__n_components': 2, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.54118, std: 0.10783, params: {'pca__n_components': 2, 'rf__min_samples_leaf': 20, 'rf__n_estimators': 100, 'rf__n_jobs': -1},
 mean: 0.36471, std: 0.14600, params: {'pca__n_components

In [33]:
metrics.accuracy_score(y_test, gridsearchcv.best_estimator_.predict(X_test))

0.6216216216216216